In [1]:
import pandas as pd
import rasterio
import numpy as np
import glob
from lab5functions import slopeAspect, reclassAspect, reclassByHisto

In [10]:
#creating zonal stats function
def getZonalStatsTable(value_raster, zone_raster):
    '''
    Function that calculates the mean, standard deviation, min, max and count of the value_raster based on
    distinct zones within the zonal_raster. Numpy as np and Pandas as pd are required.
    '''
    distinct_zones = np.unique(zone_raster)
    df = pd.DataFrame(columns=['zone', 'mean', 'max', 'min', 'stdev', 'count'])
    df['zone'] = distinct_zones[~np.isnan(distinct_zones)].astype(int)
    for zone in df['zone']:
        inZone = value_raster[zone_raster == zone]
        df.at[df['zone'] == zone,'mean'] = inZone.mean()
        df.at[df['zone'] == zone,'max'] = inZone.max()
        df.at[df['zone'] == zone,'min'] = inZone.min()
        df.at[df['zone'] == zone,'stdev'] = inZone.std()
        df.at[df['zone'] == zone,'count'] = (zone_raster == zone).sum()
        
    return df

In [11]:
#set up data and data directories
TIF_DATA_DIR = r'C:\Users\delaney\Desktop\geog5092_workspace\lab5\data'
LANDSAT_DATA_DIR = r'C:\Users\delaney\Desktop\geog5092_workspace\lab5\data\L5_big_elk'
OUT_DATA_DIR = r'C:\Users\delaney\Desktop\geog5092_workspace\lab5\output_data'

tifs = sorted(glob.glob(TIF_DATA_DIR + r'\*.tif'))
landsats = sorted(glob.glob(LANDSAT_DATA_DIR + r'\*tif'))

In [12]:
#opening/reading DEM
with rasterio.open(tifs[0]) as DEM:
    DEM_cell_size = DEM.transform[0]
    DEM_array = DEM.read(1)
    
    #calculating slope and aspect
    slp, aspect = slopeAspect(DEM_array, DEM_cell_size)
    
    #reclassifying aspect to cardinal directions
    reclass_aspect = reclassAspect(aspect)
    
    #reclassify the slope into 10 bins
    reclass_slope = reclassByHisto(slp, 10)

In [13]:
#reading in the fire perimeter data
with rasterio.open(tifs[1]) as fire_perimeter:
    fire_area = fire_perimeter.read(1)

#prepping data for calculations
band3s = []
band4s = []
RR_arrays = []

for image in landsats:
    if 'B3' in image:
        band3s.append(image)
    if 'B4' in image:
        band4s.append(image)

index = 0
yr = 2002
while index < 10:
    with rasterio.open(band3s[index]) as band3:
        band3_array = band3.read(1)
    with rasterio.open(band4s[index]) as band4:
        band4_array = band4.read(1)
    index += 1
    
    #NDVI calculations
    NDVI = (band4_array - band3_array) / (band4_array + band3_array)
    mean_healthy_NDVI = NDVI[fire_area == 2].mean()
    
    #recovery ratio calculation
    RR = NDVI / mean_healthy_NDVI
    RR_arrays.append(RR.flatten())
    
    #final print statements
    print(f'Mean recovery ratio for the year {yr}:', round(RR[fire_area == 1].mean(), 3))
    yr += 1

Mean recovery ratio for the year 2002: 0.411
Mean recovery ratio for the year 2003: 0.541
Mean recovery ratio for the year 2004: 0.513
Mean recovery ratio for the year 2005: 0.615
Mean recovery ratio for the year 2006: 0.716
Mean recovery ratio for the year 2007: 0.705
Mean recovery ratio for the year 2008: 0.74
Mean recovery ratio for the year 2009: 0.713
Mean recovery ratio for the year 2010: 0.585
Mean recovery ratio for the year 2011: 0.626


In [14]:
#calculating the trend of the RR
CoR = np.zeros_like(RR_arrays[0])
xs = range(10)
for pixel in range(RR_arrays[0].size):
    ys = [a[pixel] for a in RR_arrays]
    CoR[pixel] = np.polyfit(xs, ys, 1)[0]
CoR = CoR.reshape(band3_array.shape)
print('Mean coefficient of recovery across all years for the burned area:', CoR[fire_area == 1].mean())
CoR[fire_area != 1] = np.nan

Mean coefficient of recovery across all years for the burned area: 0.02179563


In [15]:
#calling zonal statistics function and creating two output csvs to out data directory
aspect_burned = np.where(fire_area == 1, reclass_aspect, np.nan)
slope_burned = np.where(fire_area == 1, reclass_slope, np.nan)

getZonalStatsTable(CoR, aspect_burned).to_csv(OUT_DATA_DIR + r'\aspect_stats.csv', index = False)
getZonalStatsTable(CoR, slope_burned).to_csv(OUT_DATA_DIR + r'\slope_stats.csv', index = False)

In [16]:
#writing the CoR array as a geotiff
template = rasterio.open(tifs[1])

with rasterio.open(OUT_DATA_DIR + r'\CoR_raster.tif', 'w',
                   driver = 'GTiff',
                   height = CoR.shape[0],
                   width = CoR.shape[1],
                   count = 1,
                   dtype = np.float32,
                   transform = template.transform,
                   crs = template.crs,
                   nodata = -99
) as out_raster:
    CoR = CoR.astype(np.float32)
    out_raster.write(CoR, indexes = 1)

In [17]:
#conclusions
print('After a wildfire event, I would expect that southwest facing slopes display better vegetation recovery compared \nto other aspects due to better sunlight exposure. However, the data suggests that recovery was higher on more \nnorthern facing slopes. Slope displayed better rates of recovery on less steep slopes, which was as expected.')

After a wildfire event, I would expect that southwest facing slopes display better vegetation recovery compared 
to other aspects due to better sunlight exposure. However, the data suggests that recovery was higher on more 
northern facing slopes. Slope displayed better rates of recovery on less steep slopes, which was as expected.
